In [6]:
###to get uses
### we'll get the first line from each para since they contain the information in most of the cases
### dont pick paras starting with 'Consult'
### we'll then apply syntactic extraction from them

###for administration, we create a dictionary

import spacy
import json
from spacy import displacy
import en_core_web_md
import csv
nlp = en_core_web_md.load()

def med_usage(doc):
    side_list=['medication','combination','symptoms','e','variety','children','weeks','protection','types','organisms','adults','kids','drugs','medications','diease','diseases','uses','people','levels','illness','illnesses','insects','type','virus','yeast','bacteria','eyes','eye','men','substance','treatment','kinds','urge','patients','patient']
    sym_set=set()
    for token in doc:
        if (token.pos_ == 'NOUN' or token.pos_ == 'PROPN') and (token.text.lower() not in side_list) :
            if token.dep_=='compound' and token.head.pos_ == 'NOUN': 
                sym_set.add(token.text+' '+token.head.text)
            elif token.dep_ in ['dobj','conj','pobj']:
                child_flag=False
                for child in token.children:
                    if child.dep_=='compound':
                        child_flag=True
                if not child_flag:
                    sym_set.add(token.text)
                    
    sym_set=list(sym_set)
    return sym_set

mode_dic={'topical':'Used as Cream','articular':'Via Injection to Joints','buccal':'Via the mouth','cap':'Apply lotion/shampoo on scalp',
       'cleanser':'Via the mouth', 'dental':'Apply to tooth/tooth pocket','drops':'Via drops','ear':'Apply to ear',
          'epidural':'Via Injection to Spine','eye':'Apply to eye via drops','eyelashes':'Apply to eye via drops','gel':'Apply to mouth',
          'implant':'Surgically placed by doctors','inhalation':'Via Inhaler','injection':'Via Injection',
          'intracameral':'Via injection into the eye cavity','intracavernosal':'Via Injection to the penis',
          'intramuscular':'Via injection into the muscles','intrathecal':'Via Injection in the spinal canal',
          'intratympanic':'Via Injection(Ear)','intrauterine':'Via a Device placed in the Uterus','intravenous':'Via Injection into the Veins',
          'intravesical':'Given into the bladder through a tube','intravitreal':'Via Injection into the eye','liquids':'Oral Liquids',
          'membrane':'Via the mouth','nasal':'Via Nasal Spray/Liquid','oil':'Apply oil to affected areas',
          'ophthalmic':'Eye medicine','oral':'Administered via the mouth','otic':'Apply to ear','pen':'Via Injection','pf':'Apply drops',
          'rectal':'Apply to rectum area','scalp':'Apply to head','subcutaneous':'Via Injection to the Skin',
          'subdermal':'Insert under the skin','sublingual':'Applied under the tongue','subretinal':'Via Injection into the eye',
          'suspension':'Apply as eye drops','topical':'Apply as ointment to affected areas/skin','transdermal':'Apply as ointment/patch on skin',
          'urethral':'Apply into the penis','vaginal':'Apply to vagina','xenaflamm':'Via the mouth'}
          
tsv_reader = csv.reader(open('/Users/sharadsharma/Documents/KG/Project/MedicineExtractedData.tsv'), delimiter='\t')


for (idx, (url,side_effect, usage)) in enumerate(tsv_reader):
    medicine_final_list=[]
    mode=''
    if usage!='' :
        use=usage.split()[0].lower()
        if use!='consult':
            usage=usage.partition('.')[0]
            usage=nlp(usage)
            medicine_final_list=med_usage(usage)
            
    r=url.split('/')[-2]
    r=r.split('-')[-1]
    if r in mode_dic:
        mode=mode_dic[r]
               
    
    dici={"url":url,"Uses":medicine_final_list,"Drug Administration":mode}
    with open('/Users/sharadsharma/Documents/KG/Project/MedicineUsageAdministration.jl', 'a+') as outfile:
        json.dump(dici, outfile)
        outfile.write('\n')
        
    